In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from copy import copy

In [1]:
stock_df = pd.read_csv ("../input/stockcsv/stock.csv")
stock_df.head()

In [1]:
stock_df.describe()

### Plot the Data

In [1]:
def show_plot (df,fig_title):
    df.plot (x = 'Date', figsize = (15,8), title = fig_title, linewidth = 2)
    plt.grid()
    plt.show()

In [1]:
show_plot (stock_df, 'Raw stock prices (Without Normilization)')

### Normalizes the Data

In [1]:
stock_df

In [1]:
def normalized (df):
    x = df.copy ()
    for i in x.columns [1:]:
        x[i] = x[i]/ x[i][0]
    return x

In [1]:
normalized(stock_df)

In [1]:
show_plot(normalized(stock_df) , 'Normalized the stock price')

### Perform the interactive data

In [1]:
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go

In [1]:
def interactive_plot (df, title):
    fig = px.line(title = title)
    for i in df.columns[1:]:
        fig.add_scatter (x = df['Date'] ,y = df[i] ,name = i )
        
    fig.show()

In [1]:
interactive_plot (stock_df ,"Prices")

### Calculate indivudal stock daily return

In [1]:
df = stock_df['sp500']
df

In [1]:
df_daily_return = df.copy()

for j in range (1,len(df)):
    df_daily_return [j] = ((df[j]-df[j-1]) / df[j-1] )* 100
    
df_daily_return [0] = 0
df_daily_return

### Calculate multiple stock daily return

In [1]:
def daily_return (df):
    df_daily_retun = df.copy()
    
    
    for i in df.columns[1:]:
        
        for j in range (1,len(df)):
            df_daily_retun[i][j] = ((df[i][j] - df[i][j-1]) / df[i][j-1]) * 100
            
            
        df_daily_retun[i][0] = 0
        
    return df_daily_retun
    
    

In [1]:
stock_daily_return = daily_return (stock_df)
stock_daily_return

In [1]:
show_plot (stock_daily_return , "Stock_daily_return")

In [1]:
interactive_plot(stock_daily_return , "Interactive_plot")

### Calculate the correlation of Daily Return 

In [1]:
cm = stock_daily_return.drop (columns = ['Date']).corr()

In [1]:
plt.figure (figsize = (15,8))
sns.heatmap (cm,annot = True)

### Plot the Histrogram on Daily Return

In [1]:
stock_daily_return.hist(figsize =(15,8),bins = 20);

In [1]:
df_hist = stock_daily_return.copy()
df_hist = df_hist.drop(columns = ['Date']) 

data = []

for i in df_hist.columns:
    data.append (stock_daily_return[i].values)
    
data



In [1]:
fig = ff.create_distplot(data,df_hist.columns)
fig.show()

### Perform the Random asset allocation and Calculate the Portfolio of Daily Return

#### Create the Weight Normalize The stock values****

In [1]:
np.random.seed (101)
weights = np.array(np.random.random(9))
weights

In [1]:
weights = weights / np.sum (weights)
weights

In [1]:
df_protfolio = normalized(stock_df)
df_protfolio
df_protfolio.columns [1:]

#### Note that enumarate return the values

In [1]:
for counter, stock in enumerate (df_protfolio.columns [1:]):
    df_protfolio[stock] = df_protfolio [stock] * weights[counter]
    df_protfolio [stock] = df_protfolio [stock]*1000000
    
df_protfolio

In [1]:
weights

### Portfolio allocation = Daily Return/ Worth Calculation

In [1]:
def portfolio_allocation (df,weights):
    df_protfolio = df.copy()
    df_protfolio = normalized(df_protfolio)
    
    for counter, stock in enumerate (df_protfolio.columns [1:]):
        df_protfolio[stock] = df_protfolio [stock] * weights[counter]
        df_protfolio [stock] = df_protfolio [stock]*1000000
        
    df_protfolio ['Portfolio daily Worth $'] = df_protfolio [df_protfolio != 'Date'].sum(axis = 1)
    
    df_protfolio ['Daily % Return'] = 0.0000
    
    for i in range (1,len(stock_df)):
        df_protfolio ['Daily % Return'][i] = ( ( df_protfolio ['Portfolio daily Worth $'][i] -  df_protfolio ['Portfolio daily Worth $'][i-1])
                                              /  df_protfolio ['Portfolio daily Worth $'][i-1]) * 100
        
        
        
    df_protfolio ['Daily % Return'][0] = 0
    
    return df_protfolio
    

In [1]:
df_protfolio = portfolio_allocation(stock_df , weights)
df_protfolio

## Perform Portfolio Data Visulization

### Plot of daily visulation

In [1]:
fig = px.line (x= df_protfolio.Date , y = df_protfolio['Daily % Return'] , title = 'Portfolio Daily % Return')
fig.show()

### plot of all stocks

In [1]:
interactive_plot(df_protfolio.drop (['Portfolio daily Worth $','Daily % Return'] , axis = 1),title = 'Portfolio Individule Stock  Worth in $ Overtime ')

### print Out Histogram and Daily return

In [1]:
fig = px.histogram (df_protfolio,x = "Daily % Return")
fig.show()

## Return the code with various value and  Visulize the final value

In [1]:
fig = px.line(x = df_protfolio['Date'], y = df_protfolio['Portfolio daily Worth $'], title = 'Portfolio Overall Worth in $')
fig.show()

# Calculate Portfolio Statistical Metrics(Cummulative Return, Sharp Ratio and Average Daily Return)

## Cummulative Return of the Portfolio

In [1]:
df_protfolio 

In [1]:
df_protfolio ['Portfolio daily Worth $'][-1:]

In [1]:
cummulative_return = ((df_protfolio['Portfolio daily Worth $'][-1:] - df_protfolio ['Portfolio daily Worth $'][0]) / df_protfolio ['Portfolio daily Worth $'][0]) * 100
print(cummulative_return)

## Calculate the Portfolio Standard Deviation

In [1]:
stnd_div = df_protfolio['Daily % Return'].std()
stnd_div

## Calculate the Average Daily Return

In [1]:
average = df_protfolio['Daily % Return'].mean()
average

## Calculate the Sharp Ratio 

In [1]:
sharp_ratio = (average / stnd_div) * np.sqrt(252)
sharp_ratio

# Capital Asset Pricing Model

### Understanding the Capital Asset Pricing Model (CAPM)
The formula for calculating the expected return of an asset given its risk is as follows:

\begin{aligned} &ER_i = R_f + \beta_i ( ER_m - R_f ) \\ &\textbf{where:} \\ &ER_i = \text{expected return of investment} \\ &R_f = \text{risk-free rate} \\ &\beta_i = \text{beta of the investment} \\ &(ER_m - R_f) = \text{market risk premium} \\ \end{aligned} 
​	  
ER 
i
​	 =R 
f
​	 +β 
i
​	 (ER 
m
​	 −R 
f
​	 )
where:
ER 
i
​	 =expected return of investment
R 
f
​	 =risk-free rate
β 
i
​	 =beta of the investment
(ER 
m
​	 −R 
f
​	 )=market risk premium
​	


In [1]:
stock_df

In [1]:
stock_daily_return['sp500']

In [1]:
## Fit a Polinomial line b2in selected stock price and sp500
beta,alpha = np.polyfit (stock_daily_return['AAPL'] , stock_daily_return ['sp500'] , 1)
beta,alpha



In [1]:
# Lets get plot on scatter plot and straight line plot
stock_daily_return.plot (kind = 'scatter' , x = 'sp500' , y = 'AAPL')
## Straight Line equation y = beta * rm + alpha
plt.plot(stock_daily_return['sp500'],beta * stock_daily_return['sp500'] + alpha , '-' , color = 'g')

## Apply CAPM formula in individual stock 

In [1]:
# Lets calculate the annualize rate of return on s&p500
# Note that out of 365 days, stock exchange are close for  113 days due to  holiday and weekend 
rm = stock_daily_return ['sp500'].mean() * 252
rm

In [1]:
# Assume the risk free rate is 0
# Also we can take 10 years US govt bond for  risk free rate
rf = 0
# Calculate  return for any security (APPL) using CAPM
ER_AAPL = rf + (beta * (rm - rf))
ER_AAPL

## CALCULATE BETA FOR ALL STOCKS

In [1]:
beta = {}
alpha = {}
for i in stock_daily_return.columns:
    if i != 'Date' and i != 'sp500':
        
        stock_daily_return.plot (kind = 'scatter', x = 'sp500', y = i )
        b,a = np.polyfit(stock_daily_return['sp500'] , stock_daily_return [i] , 1)
        plt.plot (stock_daily_return['sp500'], b * stock_daily_return['sp500'] + a , '-' , color = 'red')
        
        beta[i] = b
        alpha[i] = a
        
        plt.show()

In [1]:
### Lets view beta for every stock
beta

In [1]:
### Lets view alpha for every stock
alpha

### using the plotly express , plot, interactive the plot showing s&p500 vs every stock
### plot the best fil straight line for alpha and bita value

In [1]:
for i in stock_daily_return.columns:
    if i != 'Date' and i != 'sp500':
        
        fig = px.scatter (stock_daily_return,x = 'sp500', y = i , title = i)
        b,a = np.polyfit(stock_daily_return['sp500'], stock_daily_return[i], 1)
        fig.add_scatter (x = stock_daily_return['sp500'] , y = b * stock_daily_return['sp500'] + a)
        fig.show()